In [1]:
from utils.audio_dataset_utils import train_val_dataset
from network_models.soundsream_models_and_utils.ss_model_dim_red import SSDimRedModel
from network_models.soundsream_models_and_utils.ss_model_conv import SSConvModel3Sec
from network_models.soundsream_models_and_utils.ss_encoded_dataset import ss_encoded_dataset_full
import torch
from torch.utils.data import DataLoader

batch_size = 6

data_set= ss_encoded_dataset_full(
    csvPath="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/allEncodings.pkl", device="cuda")


modelDimRed = SSDimRedModel().cuda()
modelConv = SSConvModel3Sec(xSize=512, ySize=175).cuda()


trainDS, testDs = train_val_dataset(data_set, val_split=0.1, seed=100)

2023-02-09 14:19:42 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-02-09 14:19:42.840181: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-09 14:19:43.263268: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-09 14:19:43.263326: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-09 14:19:43.263331: W tensorflow/compiler/tf

In [2]:
modelConv.load_state_dict(torch.load("../content/soundstream/experiments/nr2Run_Nr_0_conv/emo_reco_best_ep216_acc_62"))
modelDimRed.load_state_dict(torch.load("../content/soundstream/experiments/nr2Run_Nr_0_dimred/emo_reco_500.pth"))

<All keys matched successfully>

In [3]:
import gc


def evaluate(device, model, dataset):
    loader = DataLoader(dataset, shuffle=False, batch_size=batch_size)
    true, preds = [], []
    with torch.no_grad():
        for batch, (X, z) in enumerate(loader):
            X, z = X.to(device),  z.to(device)
            pred = model(X)
            labels = [torch.squeeze(a.nonzero()).item() for a in z]
            true = true + labels

            preds = preds + pred.argmax(1).cpu().numpy().tolist()
            #preds.append(pred.argmax(1))
            if batch % 10 == 0:
                print(f"{batch*batch_size} of {len(dataset)}")

    return true, preds

In [4]:
from utils.eval_utils import classificationReport, confusion_matrix

trueConv, predsConv = evaluate("cuda", modelConv, testDs)
trueDimRed, predsDimRed = evaluate("cuda", modelDimRed, testDs)

print("------------------ Conv: ----------------")
classificationReport(trueConv, predsConv, data_set.encoded_dataset.label_list)
confusion_matrix(trueConv, predsConv, data_set.encoded_dataset.label_list)
print("------------------ DimRed: ----------------")
classificationReport(trueDimRed, predsDimRed, data_set.encoded_dataset.label_list)
confusion_matrix(trueDimRed, predsDimRed, data_set.encoded_dataset.label_list)

/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/venv/lib/python3.10/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


0 of 610
60 of 610
120 of 610
180 of 610
240 of 610
300 of 610
360 of 610
420 of 610
480 of 610
540 of 610
600 of 610
0 of 610
60 of 610
120 of 610
180 of 610
240 of 610
300 of 610
360 of 610
420 of 610
480 of 610
540 of 610
600 of 610
------------------ Conv: ----------------
           accuracy  precision  recall   support
   angry     0.595     0.640     0.576    99
 disgust     0.595     0.582     0.729    107
    fear     0.595     0.557     0.487    80
   happy     0.595     0.500     0.481    77
 neutral     0.595     0.758     0.726    95
     sad     0.595     0.697     0.582    91
surprise     0.595     0.395     0.492    61
                                          610

 
     avg     0.595     0.590     0.582    

                   angry     disgust        fear       happy     neutral         sad    surprise    support
       angry       0.576       0.141       0.040       0.121       0.030       0.030       0.061         99
     disgust       0.065       0.729       0.028